In [1]:
from imblearn.over_sampling import SMOTE
from scipy import stats
from sklearn import metrics
from sklearn.metrics import confusion_matrix, roc_auc_score, roc_curve,f1_score
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
import numpy
import pandas as pd
import pickle
import time
from bayes_opt import BayesianOptimization
from sklearn.model_selection import GridSearchCV
import xgboost as xgb
import warnings
warnings.filterwarnings("ignore")

#### mof

In [2]:
# dataset with FRAX CRFs and GRS
with open('ready_whi_sp23', 'rb') as file_handler:
    data = pickle.load(file_handler)
    X1, Y1 = data.get('X', []).values, data.get('Y', []).values
x_train1, x_test1, y_train1, y_test1 = train_test_split(X1, Y1, test_size=0.2,random_state=98)
sm = SMOTE(random_state=2)
x_train_s1, y_train_s1 = sm.fit_resample(x_train1, y_train1)

In [3]:
# dataset with FRAX CRFs (no grs)
with open('ready_whi_sp23', 'rb') as file_handler:
    data = pickle.load(file_handler)
    X2, Y2 = data.get('X_nogrs', []).values, data.get('Y', []).values
x_train2, x_test2, y_train2, y_test2 = train_test_split(X2, Y2, test_size=0.2,random_state=98)
sm = SMOTE(random_state=2)
x_train_s2, y_train_s2 = sm.fit_resample(x_train2, y_train2)

In [4]:
# Bayesian optimization for Model 4 (FRAX CRFs + GRS)
with warnings.catch_warnings():
    warnings.filterwarnings('ignore')
def xgb_classifier(n_estimators, max_depth, reg_alpha,reg_lambda, gamma, learning_rate, subsample):
    params = {"booster": 'gbtree',
              "objective" : "binary:logistic",
              "eval_metric" : "auc", 
              "n_estimators": int(n_estimators),
              "max_depth" : int(max_depth),
              "reg_alpha" : reg_alpha,
              "reg_lambda" : reg_lambda,
              "gamma": gamma,
              "learning_rate" : learning_rate,
              'subsample': subsample          
              }
    train_data = xgb.DMatrix(x_train_s1, y_train_s1) # grs
    cv_result = xgb.cv(params,
                       train_data,
                       nfold=10)
    return cv_result['test-auc-mean'].iloc[-1]
def optimize_xgb(data, targets):
    xgbBO = BayesianOptimization(xgb_classifier, 
                             {  
                                'n_estimators': (20, 400),
                                'max_depth': (1, 8),
                                'reg_alpha': (1, 100),
                                'reg_lambda': (1, 100),
                                'gamma': (0.001, 10),
                                'learning_rate': (0.01,1),
                                'subsample': (0.1,0.8) 
                              }, verbose=0)
    xgbBO.maximize(n_iter=10, init_points=2)
    print("Final result:", xgbBO.max)

if __name__ == "__main__":
    print("--- Optimizing XGBoost Tree with grs---")
    optimize_xgb(data=x_train_s1, targets=y_train_s1)

--- Optimizing XGBoost Tree with grs---
[16:42:19] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-0fc7796c793e6356f-1/xgboost/xgboost-ci-windows/src/learner.cc:767: 
Parameters: { "n_estimators" } are not used.

[16:42:19] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-0fc7796c793e6356f-1/xgboost/xgboost-ci-windows/src/learner.cc:767: 
Parameters: { "n_estimators" } are not used.

[16:42:19] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-0fc7796c793e6356f-1/xgboost/xgboost-ci-windows/src/learner.cc:767: 
Parameters: { "n_estimators" } are not used.

[16:42:19] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-0fc7796c793e6356f-1/xgboost/xgboost-ci-windows/src/learner.cc:767: 
Parameters: { "n_estimators" } are not used.

[16:42:19] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-0fc7796c793e6356f-1/xgboost/xgboost-ci-windows/src/learner.cc:76

[16:42:50] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-0fc7796c793e6356f-1/xgboost/xgboost-ci-windows/src/learner.cc:767: 
Parameters: { "n_estimators" } are not used.

[16:42:50] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-0fc7796c793e6356f-1/xgboost/xgboost-ci-windows/src/learner.cc:767: 
Parameters: { "n_estimators" } are not used.

[16:42:50] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-0fc7796c793e6356f-1/xgboost/xgboost-ci-windows/src/learner.cc:767: 
Parameters: { "n_estimators" } are not used.

[16:42:50] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-0fc7796c793e6356f-1/xgboost/xgboost-ci-windows/src/learner.cc:767: 
Parameters: { "n_estimators" } are not used.

[16:42:50] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-0fc7796c793e6356f-1/xgboost/xgboost-ci-windows/src/learner.cc:767: 
Parameters: { "n_estimators" } are n

[16:43:17] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-0fc7796c793e6356f-1/xgboost/xgboost-ci-windows/src/learner.cc:767: 
Parameters: { "n_estimators" } are not used.

[16:43:17] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-0fc7796c793e6356f-1/xgboost/xgboost-ci-windows/src/learner.cc:767: 
Parameters: { "n_estimators" } are not used.

[16:43:17] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-0fc7796c793e6356f-1/xgboost/xgboost-ci-windows/src/learner.cc:767: 
Parameters: { "n_estimators" } are not used.

[16:43:17] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-0fc7796c793e6356f-1/xgboost/xgboost-ci-windows/src/learner.cc:767: 
Parameters: { "n_estimators" } are not used.

[16:43:24] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-0fc7796c793e6356f-1/xgboost/xgboost-ci-windows/src/learner.cc:767: 
Parameters: { "n_estimators" } are n

In [5]:
# Bayesian optimization Model 2 (FRAX CRFs)
with warnings.catch_warnings():
    warnings.filterwarnings('ignore')
def xgb_classifier(n_estimators, max_depth, reg_alpha,reg_lambda, gamma, learning_rate, subsample):
    params = {"booster": 'gbtree',
              "objective" : "binary:logistic",
              "eval_metric" : "auc", 
              "n_estimators": int(n_estimators),
              "max_depth" : int(max_depth),
              "reg_alpha" : reg_alpha,
              "reg_lambda" : reg_lambda,
              "gamma": gamma,
              "learning_rate" : learning_rate,
              'subsample': subsample          
              }
    train_data = xgb.DMatrix(x_train_s2, y_train_s2) # grs
    cv_result = xgb.cv(params,
                       train_data,
                       nfold=10)
    return cv_result['test-auc-mean'].iloc[-1]
def optimize_xgb(data, targets):
    xgbBO = BayesianOptimization(xgb_classifier, 
                             {  
                                'n_estimators': (20, 400),
                                'max_depth': (1, 8),
                                'reg_alpha': (1, 100),
                                'reg_lambda': (1, 100),
                                'gamma': (0.001, 10),
                                'learning_rate': (0.01,1),
                                'subsample': (0.1,0.8) 
                              }, verbose=0)
    xgbBO.maximize(n_iter=10, init_points=2)
    print("Final result:", xgbBO.max)

if __name__ == "__main__":
    print("--- Optimizing XGBoost Tree with grs---")
    optimize_xgb(data=x_train_s2, targets=y_train_s2)

--- Optimizing XGBoost Tree with grs---
[16:43:47] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-0fc7796c793e6356f-1/xgboost/xgboost-ci-windows/src/learner.cc:767: 
Parameters: { "n_estimators" } are not used.

[16:43:47] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-0fc7796c793e6356f-1/xgboost/xgboost-ci-windows/src/learner.cc:767: 
Parameters: { "n_estimators" } are not used.

[16:43:47] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-0fc7796c793e6356f-1/xgboost/xgboost-ci-windows/src/learner.cc:767: 
Parameters: { "n_estimators" } are not used.

[16:43:47] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-0fc7796c793e6356f-1/xgboost/xgboost-ci-windows/src/learner.cc:767: 
Parameters: { "n_estimators" } are not used.

[16:43:47] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-0fc7796c793e6356f-1/xgboost/xgboost-ci-windows/src/learner.cc:76

[16:44:10] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-0fc7796c793e6356f-1/xgboost/xgboost-ci-windows/src/learner.cc:767: 
Parameters: { "n_estimators" } are not used.

[16:44:10] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-0fc7796c793e6356f-1/xgboost/xgboost-ci-windows/src/learner.cc:767: 
Parameters: { "n_estimators" } are not used.

[16:44:10] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-0fc7796c793e6356f-1/xgboost/xgboost-ci-windows/src/learner.cc:767: 
Parameters: { "n_estimators" } are not used.

[16:44:10] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-0fc7796c793e6356f-1/xgboost/xgboost-ci-windows/src/learner.cc:767: 
Parameters: { "n_estimators" } are not used.

[16:44:10] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-0fc7796c793e6356f-1/xgboost/xgboost-ci-windows/src/learner.cc:767: 
Parameters: { "n_estimators" } are n

[16:44:33] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-0fc7796c793e6356f-1/xgboost/xgboost-ci-windows/src/learner.cc:767: 
Parameters: { "n_estimators" } are not used.

[16:44:33] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-0fc7796c793e6356f-1/xgboost/xgboost-ci-windows/src/learner.cc:767: 
Parameters: { "n_estimators" } are not used.

[16:44:38] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-0fc7796c793e6356f-1/xgboost/xgboost-ci-windows/src/learner.cc:767: 
Parameters: { "n_estimators" } are not used.

[16:44:38] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-0fc7796c793e6356f-1/xgboost/xgboost-ci-windows/src/learner.cc:767: 
Parameters: { "n_estimators" } are not used.

[16:44:38] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-0fc7796c793e6356f-1/xgboost/xgboost-ci-windows/src/learner.cc:767: 
Parameters: { "n_estimators" } are n

In [6]:
# grid search for Model 3 (FRAX CRFs + GRS)
import xgboost as xgb
parameters = {
        'n_estimators': [20, 200, 400],
        'subsample': [0.1, 0.5, 0.8],
        'learning_rate': [0.01, 0.1, 1],
        'gamma': [0.001,5,10],
        'reg_alpha': [1,10,100],
        'reg_lambda': [1,10,100],
        'max_depth': [1,5, 8],
        'random_state': [42]
     }
with warnings.catch_warnings():
    warnings.filterwarnings('ignore')
    xgb = xgb.XGBClassifier()
    model = GridSearchCV(estimator=xgb, 
                    param_grid=parameters,cv=10).fit(x_train_s1, y_train_s1)
    print(model.best_params_)

{'gamma': 0.001, 'learning_rate': 1, 'max_depth': 5, 'n_estimators': 400, 'random_state': 42, 'reg_alpha': 10, 'reg_lambda': 10, 'subsample': 0.8}


In [7]:
# grid search for Model 1 (FRAX CRFs)
import xgboost as xgb
parameters = {
        'n_estimators': [20, 200, 400],
        'subsample': [0.1, 0.5, 0.8],
        'learning_rate':[0.01, 0.1, 1],
        'gamma': [0.001,5,10],
        'reg_alpha': [1,10,100],
        'reg_lambda': [1,10,100],
        'max_depth': [1,5, 8],
        'random_state': [42]
     }
with warnings.catch_warnings():
    warnings.filterwarnings('ignore')
    xgb = xgb.XGBClassifier()
    model = GridSearchCV(estimator=xgb, 
                    param_grid=parameters,cv=10).fit(x_train_s2, y_train_s2)
    print(model.best_params_)

{'gamma': 0.1, 'learning_rate': 1, 'max_depth': 5, 'n_estimators': 400, 'random_state': 42, 'reg_alpha': 1, 'reg_lambda': 100, 'subsample': 0.8}


#### hip

In [8]:
# dataset with FRAX CRFs and GRS
with open('ready_whi_sp23_hip_py38', 'rb') as file_handler:
    data = pickle.load(file_handler)
    X1, Y1 = data.get('X', []).values, data.get('Y', []).values
x_train1, x_test1, y_train1, y_test1 = train_test_split(X1, Y1, test_size=0.2,random_state=98)
sm = SMOTE(random_state=2)
x_train_s1, y_train_s1 = sm.fit_resample(x_train1, y_train1)

In [9]:
# dataset with FRAX CRF (no grs)
with open('ready_whi_sp23_hip_py38', 'rb') as file_handler:
    data = pickle.load(file_handler)
    X2, Y2 = data.get('X_nogrs', []).values, data.get('Y', []).values
x_train2, x_test2, y_train2, y_test2 = train_test_split(X2, Y2, test_size=0.2,random_state=98)
sm = SMOTE(random_state=2)
x_train_s2, y_train_s2 = sm.fit_resample(x_train2, y_train2)

In [10]:
# Bayesian optimization for Model 4 (FRAX CRFs + GRS)
import xgboost as xgb
with warnings.catch_warnings():
    warnings.filterwarnings('ignore')
def xgb_classifier(n_estimators, max_depth, reg_alpha,reg_lambda, gamma, learning_rate, subsample):
    params = {"booster": 'gbtree',
              "objective" : "binary:logistic",
              "eval_metric" : "auc", 
              "n_estimators": int(n_estimators),
              "max_depth" : int(max_depth),
              "reg_alpha" : reg_alpha,
              "reg_lambda" : reg_lambda,
              "gamma": gamma,
              "learning_rate" : learning_rate,
              'subsample': subsample          
              }
    train_data = xgb.DMatrix(x_train_s1, y_train_s1) # grs
    cv_result = xgb.cv(params,
                       train_data,
                       nfold=10)
    return cv_result['test-auc-mean'].iloc[-1]
def optimize_xgb(data, targets):
    xgbBO = BayesianOptimization(xgb_classifier, 
                             {  
                                'n_estimators': (20, 400),
                                'max_depth': (1, 8),
                                'reg_alpha': (1, 100),
                                'reg_lambda': (1, 100),
                                'gamma': (0.001, 10),
                                'learning_rate': (0.01,1),
                                'subsample': (0.1,0.8) 
                              }, verbose=0)
    xgbBO.maximize(n_iter=10, init_points=2)
    print("Final result:", xgbBO.max)

if __name__ == "__main__":
    print("--- Optimizing XGBoost Tree with grs---")
    optimize_xgb(data=x_train_s1, targets=y_train_s1)

--- Optimizing XGBoost Tree with grs---
[16:48:00] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-0fc7796c793e6356f-1/xgboost/xgboost-ci-windows/src/learner.cc:767: 
Parameters: { "n_estimators" } are not used.

[16:48:01] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-0fc7796c793e6356f-1/xgboost/xgboost-ci-windows/src/learner.cc:767: 
Parameters: { "n_estimators" } are not used.

[16:48:01] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-0fc7796c793e6356f-1/xgboost/xgboost-ci-windows/src/learner.cc:767: 
Parameters: { "n_estimators" } are not used.

[16:48:01] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-0fc7796c793e6356f-1/xgboost/xgboost-ci-windows/src/learner.cc:767: 
Parameters: { "n_estimators" } are not used.

[16:48:01] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-0fc7796c793e6356f-1/xgboost/xgboost-ci-windows/src/learner.cc:76

[16:48:30] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-0fc7796c793e6356f-1/xgboost/xgboost-ci-windows/src/learner.cc:767: 
Parameters: { "n_estimators" } are not used.

[16:48:30] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-0fc7796c793e6356f-1/xgboost/xgboost-ci-windows/src/learner.cc:767: 
Parameters: { "n_estimators" } are not used.

[16:48:30] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-0fc7796c793e6356f-1/xgboost/xgboost-ci-windows/src/learner.cc:767: 
Parameters: { "n_estimators" } are not used.

[16:48:30] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-0fc7796c793e6356f-1/xgboost/xgboost-ci-windows/src/learner.cc:767: 
Parameters: { "n_estimators" } are not used.

[16:48:30] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-0fc7796c793e6356f-1/xgboost/xgboost-ci-windows/src/learner.cc:767: 
Parameters: { "n_estimators" } are n

[16:48:58] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-0fc7796c793e6356f-1/xgboost/xgboost-ci-windows/src/learner.cc:767: 
Parameters: { "n_estimators" } are not used.

[16:48:58] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-0fc7796c793e6356f-1/xgboost/xgboost-ci-windows/src/learner.cc:767: 
Parameters: { "n_estimators" } are not used.

[16:48:58] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-0fc7796c793e6356f-1/xgboost/xgboost-ci-windows/src/learner.cc:767: 
Parameters: { "n_estimators" } are not used.

[16:48:58] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-0fc7796c793e6356f-1/xgboost/xgboost-ci-windows/src/learner.cc:767: 
Parameters: { "n_estimators" } are not used.

[16:49:04] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-0fc7796c793e6356f-1/xgboost/xgboost-ci-windows/src/learner.cc:767: 
Parameters: { "n_estimators" } are n

In [11]:
# Bayesian optimization for Model 2 (FRAX CRFs)
import xgboost as xgb
with warnings.catch_warnings():
    warnings.filterwarnings('ignore')
def xgb_classifier(n_estimators, max_depth, reg_alpha,reg_lambda, gamma, learning_rate, subsample):
    params = {"booster": 'gbtree',
              "objective" : "binary:logistic",
              "eval_metric" : "auc", 
              "n_estimators": int(n_estimators),
              "max_depth" : int(max_depth),
              "reg_alpha" : reg_alpha,
              "reg_lambda" : reg_lambda,
              "gamma": gamma,
              "learning_rate" : learning_rate,
              'subsample': subsample          
              }
    train_data = xgb.DMatrix(x_train_s2, y_train_s2) # grs
    cv_result = xgb.cv(params,
                       train_data,
                       nfold=10)
    return cv_result['test-auc-mean'].iloc[-1]
def optimize_xgb(data, targets):
    xgbBO = BayesianOptimization(xgb_classifier, 
                             {  
                                'n_estimators': (20, 400),
                                'max_depth': (1, 8),
                                'reg_alpha': (1, 100),
                                'reg_lambda': (1, 100),
                                'gamma': (0.001, 10),
                                'learning_rate':(0.01,1),
                                'subsample': (0.1,0.8) 
                              }, verbose=0)   
    
    xgbBO.maximize(n_iter=10, init_points=2)
    print("Final result:", xgbBO.max)

if __name__ == "__main__":
    print("--- Optimizing XGBoost Tree with grs---")
    optimize_xgb(data=x_train_s2, targets=y_train_s2)

--- Optimizing XGBoost Tree with grs---
[16:49:28] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-0fc7796c793e6356f-1/xgboost/xgboost-ci-windows/src/learner.cc:767: 
Parameters: { "n_estimators" } are not used.

[16:49:28] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-0fc7796c793e6356f-1/xgboost/xgboost-ci-windows/src/learner.cc:767: 
Parameters: { "n_estimators" } are not used.

[16:49:28] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-0fc7796c793e6356f-1/xgboost/xgboost-ci-windows/src/learner.cc:767: 
Parameters: { "n_estimators" } are not used.

[16:49:28] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-0fc7796c793e6356f-1/xgboost/xgboost-ci-windows/src/learner.cc:767: 
Parameters: { "n_estimators" } are not used.

[16:49:28] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-0fc7796c793e6356f-1/xgboost/xgboost-ci-windows/src/learner.cc:76

[16:49:51] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-0fc7796c793e6356f-1/xgboost/xgboost-ci-windows/src/learner.cc:767: 
Parameters: { "n_estimators" } are not used.

[16:49:51] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-0fc7796c793e6356f-1/xgboost/xgboost-ci-windows/src/learner.cc:767: 
Parameters: { "n_estimators" } are not used.

[16:49:51] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-0fc7796c793e6356f-1/xgboost/xgboost-ci-windows/src/learner.cc:767: 
Parameters: { "n_estimators" } are not used.

[16:49:51] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-0fc7796c793e6356f-1/xgboost/xgboost-ci-windows/src/learner.cc:767: 
Parameters: { "n_estimators" } are not used.

[16:49:51] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-0fc7796c793e6356f-1/xgboost/xgboost-ci-windows/src/learner.cc:767: 
Parameters: { "n_estimators" } are n

[16:50:15] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-0fc7796c793e6356f-1/xgboost/xgboost-ci-windows/src/learner.cc:767: 
Parameters: { "n_estimators" } are not used.

[16:50:15] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-0fc7796c793e6356f-1/xgboost/xgboost-ci-windows/src/learner.cc:767: 
Parameters: { "n_estimators" } are not used.

[16:50:21] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-0fc7796c793e6356f-1/xgboost/xgboost-ci-windows/src/learner.cc:767: 
Parameters: { "n_estimators" } are not used.

[16:50:21] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-0fc7796c793e6356f-1/xgboost/xgboost-ci-windows/src/learner.cc:767: 
Parameters: { "n_estimators" } are not used.

[16:50:21] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-0fc7796c793e6356f-1/xgboost/xgboost-ci-windows/src/learner.cc:767: 
Parameters: { "n_estimators" } are n

In [12]:
# grid search for Model 3 (FRAX CRFs + GRS)
import xgboost as xgb
parameters = {
        'n_estimators': [20, 200, 400],
        'subsample': [0.1, 0.5, 0.8],
        'learning_rate': [0.01, 0.1, 1],
        'gamma': [0.001,5,10],
        'reg_alpha': [1,10,100],
        'reg_lambda': [1,10,100],
        'max_depth': [1,5, 8],
        'random_state': [42]
     }
with warnings.catch_warnings():
    warnings.filterwarnings('ignore')
    xgb = xgb.XGBClassifier()
    model = GridSearchCV(estimator=xgb, 
                    param_grid=parameters,cv=10).fit(x_train_s1, y_train_s1)
    print(model.best_params_)

{'gamma': 0.001, 'learning_rate': 1, 'max_depth': 5, 'n_estimators': 400, 'random_state': 42, 'reg_alpha': 10, 'reg_lambda': 10, 'subsample': 0.8}


In [13]:
# grid search for Model 1 (FRAX CRFs)
import xgboost as xgb
parameters = {
        'n_estimators': [20, 200, 400],
        'subsample': [0.1, 0.5, 0.8],
        'learning_rate': [0.01, 0.1, 1],
        'gamma': [0.001,5,10],
        'reg_alpha': [1,10,100],
        'reg_lambda': [1,10,100],
        'max_depth': [1,5, 8],
        'random_state': [42]
     }
with warnings.catch_warnings():
    warnings.filterwarnings('ignore')
    xgb = xgb.XGBClassifier()
    model = GridSearchCV(estimator=xgb, 
                    param_grid=parameters,cv=10).fit(x_train_s2, y_train_s2)
    print(model.best_params_)

{'gamma': 0.1, 'learning_rate': 1, 'max_depth': 5, 'n_estimators': 400, 'random_state': 42, 'reg_alpha': 1, 'reg_lambda': 100, 'subsample': 0.5}
